In [1]:
import pandas as pd
import numpy as np
import os
import folium
from folium import plugins

In [2]:
df = pd.read_csv('./data/Serie_Total2016_ext.csv', parse_dates=['timestamp'])
df.head()

,element_key,latitude,longitude,timestamp,paid_parking_area,day_year,occupation_perc,prcp,tmax,tmin,...,poi,baseball,tennis,basket,soccer,event,no2,co,pm2_5,o3
0,1001,47.602862,-122.334703,2016-01-02 08:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
1,1001,47.602862,-122.334703,2016-01-02 09:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
2,1001,47.602862,-122.334703,2016-01-02 10:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
3,1001,47.602862,-122.334703,2016-01-02 11:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
4,1001,47.602862,-122.334703,2016-01-02 12:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0


In [28]:
# Selecciono un distrito y un periodo de tiempo (mes Enero)
sub_df = df.loc[(df.paid_parking_area == 'Belltown') & (df.timestamp.dt.month == 1)][[
    'latitude','longitude','timestamp','occupation_perc']]

In [29]:
sub_df.occupation_perc = 1 - sub_df.occupation_perc/100 # para que aparezcan en rojo los poco ocupados
sub_df = sub_df.sort_values(by=['timestamp'])
sub_df.head()

,latitude,longitude,timestamp,occupation_perc
28620,47.610756,-122.341899,2016-01-02 08:00:00,1.0000
2074728,47.613513,-122.342551,2016-01-02 08:00:00,0.8824
2078352,47.614114,-122.343973,2016-01-02 08:00:00,1.0000
2081976,47.614400,-122.344075,2016-01-02 08:00:00,0.8400
2085612,47.615678,-122.346619,2016-01-02 08:00:00,1.0000


In [30]:
sub_df.tail()

,latitude,longitude,timestamp,occupation_perc
844943,47.613176,-122.339752,2016-01-30 19:00:00,1.0000
3213419,47.614198,-122.345309,2016-01-30 19:00:00,0.7500
2661563,47.610979,-122.344277,2016-01-30 19:00:00,0.8889
469259,47.616665,-122.350965,2016-01-30 19:00:00,0.9444
3825203,47.611903,-122.347463,2016-01-30 19:00:00,0.9333


## HeatMapWithTime

In [37]:
from folium.plugins import HeatMapWithTime
from datetime import datetime
 
timestamp_values = sub_df.timestamp.unique()

data = [[[row['latitude'], row['longitude'], row['occupation_perc']] 
                for index, row in sub_df[sub_df.timestamp == tv].iterrows()] 
                 for tv in timestamp_values]

loc_lat = sub_df.latitude.mean()
loc_long = sub_df.longitude.mean()
m = folium.Map(location=[loc_lat, loc_long], tiles='cartodbpositron', zoom_start=15)
# tiles = 'openstreetmap

time_index = sub_df.timestamp.unique().astype(datetime)/1000000000
ti = [datetime.utcfromtimestamp(i).strftime('%d-%m-%Y, %H:%M') for i in time_index]

hm = plugins.HeatMapWithTime(data, ti)

m.add_child(hm)

m